In [3]:

import urllib.request
import json
import pymongo

# get the bbc rss feed of news stories and connect to it
earthquake_url = "http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson"

try:
	response = urllib.request.urlopen(earthquake_url)
except urllib.error.URLError as e:
    if hasattr(e, 'reason'):
        print('We failed to reach a server.')
        print('Reason: ', e.reason)
    elif hasattr(e, 'code'):
        print('The server couldn\'t fulfill the request.')
        print('Error code: ', e.code)
else:
	# the url request was successful - convert the response to a string
	json_string = response.read().decode('utf-8')

	# the json package loads() converts the string to python dictionaries and lists
	eq_json = json.loads(json_string)
	
	# from the json dictionary we get the title to print
	title = eq_json['metadata']['title']
	print('Collected data from', title)
	#  and we get the list of earthquakes
	quakelist = eq_json['features']

	# Connection to Mongo DB
	try:
	    client=pymongo.MongoClient('localhost', 27017)
	    print ("Connected successfully!!!")
	except pymongo.errors.ConnectionFailure as e:
	    print ("Could not connect to MongoDB: %s" % e )
	else:

		# use database named usgs or create it if not there already
		eqdb = client.usgs
		# create collection named earthquakes or create it if not there already
		quakecoll = eqdb.earthquakes
		# add all the earthquakes to the list
		quakecoll.insert_many(quakelist)
		print("Added", len(quakelist), "to earthquakes collection in usgs database")
        		# close the database connection
		client.close()

Collected data from USGS Significant Earthquakes, Past Month
Connected successfully!!!
Added 23 to earthquakes collection in usgs database


In [4]:
docs = quakecoll.find()
a = list(docs)

places = []
times = []
for i in range(0,len(a)):
    places.append((a[i]['properties']['place']))
    times.append((a[i]['properties']['time']))

for i,j in zip(places,times):
    print('Location-> {}----- Time-> {}'.format(i,j))

Location-> 45km SSE of Tanaga Volcano, Alaska----- Time-> 1534995315950
Location-> 265km WNW of Bandon, Oregon----- Time-> 1534930307050
Location-> 79km E of Lakatoro, Vanuatu----- Time-> 1534890747010
Location-> 30km NE of Rio Caribe, Venezuela----- Time-> 1534887106070
Location-> 2km S of Belanting, Indonesia----- Time-> 1534690588090
Location-> 282km ESE of Lambasa, Fiji----- Time-> 1534652939760
Location-> 6km NE of Sembalunlawang, Indonesia----- Time-> 1534651821570
Location-> 259km NNE of Ndoi Island, Fiji----- Time-> 1534638182740
Location-> 280km NNE of Ndoi Island, Fiji----- Time-> 1534637977970
Location-> 14km N of Golfito, Costa Rica----- Time-> 1534548144900
Location-> 109km NNW of Kampungbajo, Indonesia----- Time-> 1534520102070
Location-> 250km SE of Iwo Jima, Japan----- Time-> 1534443773350
Location-> 50km S of Tanaga Volcano, Alaska----- Time-> 1534370214780
Location-> 7km NE of Aguanga, CA----- Time-> 1534296266310
Location-> 90km SW of Kaktovik, Alaska----- Time-> 153

In [5]:
a 

[{'_id': ObjectId('5b7eeb8157d546645c21eb17'),
  'geometry': {'coordinates': [-177.881, 51.5106, 43.91], 'type': 'Point'},
  'id': 'us1000gfvg',
  'properties': {'alert': 'green',
   'cdi': None,
   'code': '1000gfvg',
   'detail': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/us1000gfvg.geojson',
   'dmin': 0.567,
   'felt': None,
   'gap': 67,
   'ids': ',at00pdw9ys,ak20156243,us1000gfvg,',
   'mag': 6.3,
   'magType': 'mww',
   'mmi': 5.84,
   'net': 'us',
   'nst': None,
   'place': '45km SSE of Tanaga Volcano, Alaska',
   'rms': 0.72,
   'sig': 611,
   'sources': ',at,ak,us,',
   'status': 'reviewed',
   'time': 1534995315950,
   'title': 'M 6.3 - 45km SSE of Tanaga Volcano, Alaska',
   'tsunami': 1,
   'type': 'earthquake',
   'types': ',geoserve,impact-link,losspager,moment-tensor,origin,phase-data,shakemap,',
   'tz': -600,
   'updated': 1535036746040,
   'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us1000gfvg'},
  'type': 'Feature'},
 {'_id': ObjectId(